In [ ]:
!pip install transformers datasets torch scikit-learn pandas openpyxl
!pip install -U transformers


In [ ]:
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Training_dataset_final01.xlsx to Training_dataset_final01.xlsx


In [ ]:
df = pd.read_excel("Training_dataset_final01.xlsx")

df.head()


,sentence,emotion
0,i was too young to understand the feeling of l...,suprise
1,i feel i cannot write elegant enough words to ...,joy
2,i feel unsure and slightly desperate and im no...,fear
3,i didn t exactly feel accepted or integrated i...,love
4,i was anyways not feeling too energetic,joy


In [ ]:
print(df['emotion'].value_counts())
print(df['emotion'].unique())


emotion
suprise       5000
joy           5000
fear          5000
love          5000
study/work    5000
anger         5000
sad           5000
Name: count, dtype: int64
['suprise' 'joy' 'fear' 'love' 'study/work' 'anger' 'sad']


In [ ]:
label2id = {
    'anger': 0,
    'fear': 1,
    'joy': 2,
    'love': 3,
    'sad': 4,
    'suprise': 5,
    'study/work': 6
}

id2label = {v: k for k, v in label2id.items()}

df['label'] = df['emotion'].map(label2id)


In [ ]:
train_texts = df['sentence']
train_labels = df['label']


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(
    list(train_texts),
    truncation=True,
    padding=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import torch

class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_encodings = tokenizer(
    list(train_texts),
    truncation=True,
    padding=True
)

train_dataset = EmotionDataset(train_encodings, train_labels)


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=7,
    id2label=id2label,
    label2id=label2id
)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2
)


`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,

)


In [ ]:
trainer.train()


Step,Training Loss
100,1.650005
200,0.967019
300,0.512649
400,0.358213
500,0.291324
600,0.287369
700,0.249933
800,0.236391
900,0.202813
1000,0.184408


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=8752, training_loss=0.15346139277139137, metrics={'train_runtime': 3076.735, 'train_samples_per_second': 45.503, 'train_steps_per_second': 2.845, 'total_flos': 6835027605000000.0, 'train_loss': 0.15346139277139137, 'epoch': 4.0})

In [ ]:
!nvidia-smi



In [ ]:
model.save_pretrained("emotion_model")
tokenizer.save_pretrained("emotion_model")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model.save_pretrained("/content/drive/MyDrive/emotion_model")
tokenizer.save_pretrained("/content/drive/MyDrive/emotion_model")


Mounted at /content/drive


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('/content/drive/MyDrive/emotion_model/tokenizer_config.json',
 '/content/drive/MyDrive/emotion_model/tokenizer.json')

In [ ]:
#!zip -r emotion_model.zip emotion_model
#from google.colab import files
#files.download("emotion_model.zip") !zip -r emotion_model.zip emotion_model
#from google.colab import files
#files.download("emotion_model.zip")



In [ ]:
''' import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

text = "i  passionate about each cow calf or bull"

inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    padding=True
)

# MOVE INPUTS TO SAME DEVICE AS MODEL
inputs = {key: value.to(device) for key, value in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)

pred = outputs.logits.argmax(dim=1).item()

print("Predicted emotional context:", id2label[pred])
'''
